# Importing Data and Libs

In [ ]:
#Pip installs
!pip install kaggle

#Import
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

import glob
import imageio
import numpy as np
import PIL
import time
import pathlib
from IPython import display

from zipfile import ZipFile
from google.colab import files

In [ ]:
#Kaggle Data Import
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download --force -d kmader/satellite-images-of-hurricane-damage

file_name = "/content/satellite-images-of-hurricane-damage.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Formatting Data and Visualizing 

In [ ]:
!rm -rf '/content/test_another'
!rm -rf '/content/validation_another'

train_no_damage_dir = os.path.join('/content/train_another/no_damage')
train_damage_dir = os.path.join('/content/train_another/damage')
test_no_damage_dir = os.path.join('/content/test/no_damage')
test_damage_dir = os.path.join('/content/test/damage')

print('total training damage images:', len(os.listdir(train_damage_dir)))
print('total training no damage images:', len(os.listdir(train_no_damage_dir)))
print('total testing damage images:', len(os.listdir(test_damage_dir)))
print('total testing no damage images:', len(os.listdir(test_no_damage_dir)))

damage_files = os.listdir(train_damage_dir)
print(damage_files[:10])

In [ ]:
%matplotlib inline
pic_index = 2
next_img = [os.path.join(train_damage_dir, fname) 
                for fname in damage_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_img):
  print(img_path)
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

In [ ]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/content/train_another/',  
        target_size=(300, 300), 
        batch_size=128,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        '/content/test/',  
        target_size=(300, 300),  
        batch_size=128,
        class_mode='binary')

#Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer = Adam(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

EPOCHS = 10
es = EarlyStopping(
    monitor='val_loss', 
    mode='min',
    patience=10
)

history = model.fit(
  train_generator,
  steps_per_epoch=int(10000/128),  
  epochs=EPOCHS,
  validation_data = validation_generator,
  validation_steps=int(2000/128),
  callbacks=[es]
)

In [ ]:
model.save("hurricane-weights.h5")

# Plotting Accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
loss, acc = model.evaluate(validation_generator, verbose=2)
print("Loss of model: " + str(loss))
print("Accuracy of model: "+ str(acc*100))